# Algorithm Tuning

This notebook examines the algorithm tuning process.

In [ ]:
from collections import OrderedDict
import pickle

In [ ]:
import seaborn as sns
import pandas as pd
import skopt
import matplotlib.pyplot as plt
from skopt.plots import *

In [ ]:
def read_opt(ds, algo):
    return skopt.load(f'data/{ds}/tuning/{algo}.opt')

In [ ]:
def opt_df(opt):
    ys = pd.Series(opt.func_vals)
    cols = OrderedDict()
    for i, dim in enumerate(opt.space):
        cols[f'p{i}'] = pd.Series([x[i] for x in opt.x_iters])
    cols['y'] = -ys
    if hasattr(opt, 'iter_time'):
        cols['time'] = pd.Series(opt.iter_time).reindex(ys.index)
    return pd.DataFrame(cols)

In [ ]:
def improvement_threshold(opt_df, col='y'):
    ymax = opt_df[col].max()
    return ymax - ymax * 0.01

## User-User CF

How does the user-user CF tuning perform?

In [ ]:
az_uu_i = read_opt('AZ', 'user-user-imp')
az_uu_e = read_opt('AZ', 'user-user')
bx_uu_i = read_opt('BX-I', 'user-user')
bx_uu_e = read_opt('BX-E', 'user-user')

BX-E UU plots:

In [ ]:
plot_convergence(bx_uu_e)

In [ ]:
plot_objective(bx_uu_e)

BX-I UU:

In [ ]:
plot_convergence(bx_uu_i)

In [ ]:
plot_objective(bx_uu_i)

## ALS

In [ ]:
bx_als_e = read_opt('BX-E', 'als')
bx_als_i = read_opt('BX-I', 'wrls')
az_als_e = read_opt('AZ', 'als')
az_als_i = read_opt('AZ', 'wrls-imp')

In [ ]:
plot_convergence(bx_als_e)

In [ ]:
plot_convergence(bx_als_i)

In [ ]:
plot_objective(bx_als_i)

In [ ]:
plot_convergence(az_als_e)

In [ ]:
plot_objective(az_als_e)

In [ ]:
plot_convergence(az_als_i)

In [ ]:
plot_objective(az_als_i)

### Evaluation Performance

In [ ]:
bxe_df = opt_df(bx_als_e)
bxi_df = opt_df(bx_als_i)

#### BX-E

In [ ]:
bxe_df.info()

In [ ]:
sns.scatterplot(x='p0', y='time', data=bxe_df)

In [ ]:
plt.axvline(improvement_threshold(bxe_df), 0, 1, color='grey')
sns.scatterplot(x='y', y='time', data=bxe_df)

In [ ]:
sns.scatterplot(x='y', y='time', data=bxe_df)
plt.vlines(bxe_df.y.max() - 0.005, 0, bxe_df.time.max())

#### BX-I

In [ ]:
bxi_df.info()

In [ ]:
sns.scatterplot(x='p0', y='time', data=bxi_df)

In [ ]:
plt.axvline(improvement_threshold(bxi_df), 0, 1, color='black')
sns.scatterplot(x='y', y='time', data=bxi_df)